## import local

In [1]:
from __future__ import print_function
__file__ = !cd .. ;pwd
__file__ = __file__[0]
__file__

'/Users/brucecottman/Documents/PROJECTS/paso'

In [2]:
import sys
from random import random
import pandas as pd
sys.path.append(__file__)
sys.path

['/Users/brucecottman/Documents/PROJECTS/paso/lessons',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python37.zip',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/lib-dynload',
 '',
 '/Users/brucecottman/.local/lib/python3.7/site-packages',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/site-packages',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/site-packages/aeosa',
 '/Users/brucecottman/anaconda3/envs/paso/lib/python3.7/site-packages/IPython/extensions',
 '/Users/brucecottman/.ipython',
 '/Users/brucecottman/Documents/PROJECTS/paso']

In [3]:
from paso.base import Paso,Log,PasoError,Param,NameToClass
from loguru import logger
session = Paso(parameters_filepath='../parameters/lesson.3.yaml').startup()
from paso.models.sklearn import SkModel
def df_City():
    from sklearn.datasets import load_boston
    boston = load_boston()
    City = pd.DataFrame(boston.data, columns=boston.feature_names)
    City['MEDV'] = boston.target
    return (City.copy())


paso 14.7.2019 17:20:25 INFO Log started
paso 14.7.2019 17:20:25 INFO ========================================
paso 14.7.2019 17:20:25 INFO Read in parameter file: ../parameters/lesson.3.yaml


ModuleNotFoundError: No module named 'paso.models.sklearn'

## Class Balancing and  Augmentating Structured Data

Structured Data Augmentation is very rarely discussed in theoretical or real-world production systems. It a secret sauce for which the recipe will be shown here. Along the way, we will introduce ``paso's`` Class Balancing tools. As was shown in the previous two articles, **paso** supports and is compatiable with ``sklearn``.



## A Quick Overview of paso

**paso** is a package written in Python and some ``C``(for speed) that was originally intended to bundle best-practices and state-of-the-art services, classes and functions for the Machine Learning and Deep Learning community. **paso** has grown beyond this to offer patterns, classes and methods you can use in your **Scikit-Learn pipelines** or custom code with or without adopting the entire **paso** package.

**Scikit-Learn pipelines** are composed of steps, each step has to be **Scikit-Learn transformer** or a custom **Scikit-Learn transformer**. The last of the pipeline can be a transformer or an estimator, where a **Scikit-Learn estimator** is a compatible model. 

**paso** has quite a few Class Balancing transformers that are custom **Scikit-Learn transformers**. Note that **paso** translates from Spanish to English to the word step.

Discussion will be divided into the following major segments:
- What Class Balancing stratagems are in **paso**.
- How to use Class Balancing to argument Fixed Structure data
- The benefits of argumenting Fixed Structure data.
- As usual the source code for all examples is available.

As we saw in [lesson-1](), we need to startup **paso** services.

In [ ]:
import numpy as np
import pandas as pd
from pandas_summary import DataFrameSummary

import warnings
warnings.filterwarnings("ignore")
import multiprocessing

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm

%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
import seaborn as sns
color = sns.color_palette()

from paso.base import Paso,Log,PasoError
from loguru import logger
session = Paso(parameters_filepath='../parameters/lesson.3.yaml').startup()

Next, we load the ``boston``data set into the ``City``dataframe. We will munge City up to show what the **paso** cleaners can do.

## paso Class for Handling Imbalanced Classes

All available class balance strategies are shown with:

In [ ]:
from paso.pre.cleaners import Class_Balance
o = Class_Balance('SMOTE')
o.classBalancers()

## Balance Toy Example Classes

Let us start with a toy dataset to class balance. we load the ``iris``data set, into the ``Flower``dataframe.  The feature ``TypeOf`` will contain the class target values.

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
Flower = pd.DataFrame(iris.data, columns = iris.feature_names )
Flower['TypeOf'] = iris.target
#logger.info(iris.DESCR)
DataFrameSummary(Flower).summary()

In [ ]:
targetFeature = 'TypeOf'
Class_BalancerKey = 'SMOTE'
X = Class_Balance(Class_BalancerKey).transform(Flower,targetFeature)

In [ ]:
DataFrameSummary(X).summary()

``Class_Balance`` did not change the ``Flower`` dataframe as the ``iris``dataset comes class balanced.  

In [ ]:
sns.set(rc={'figure.figsize':(6,4)})
sns.distplot((X['TypeOf'].astype('category').cat.codes.astype('int')+1),kde=False)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [ ]:
(Param.parameters_D['RandomForest' + '_kwargs'])

In [ ]:
NameToClass.__models__['RandomForest']
Param.parameters_D['RandomForest' + '_kwargs']

In [ ]:
from paso.models.sklearn import SkModel
o = SkModel()
o.train(X)

The argment ratio is the amount of the dataset to assign to the validation (test) dataset. In this case,
``X``and ``y`` datasets into training set(70\%) and test set(30\%). Train and predict using ``RandomForestClassifier``.

In [ ]:
def RF(X,targetFeature,ratio,epochs):
    accAcum = 0
    f1_score_Acc = 0
    precision_score_Acc = 0
    recall_score_Acc = 0
    y_predAcc = 0
    logLossAcc = 0
    y = X[targetFeature]
    X = X[X.columns.difference([targetFeature])]
    for round in range(epochs):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio,stratify=y)
        clf=RandomForestClassifier(n_estimators=100,n_jobs=-1)
        clf.fit(X_train,y_train)
        y_pred=clf.predict(X_test)
        y_predAcc += y_pred + 1
        accAcum += metrics.accuracy_score(y_test, y_pred)
        precision_score_Acc += metrics.precision_score(y_test, y_pred, average="macro")
        recall_score_Acc += metrics.recall_score(y_test, y_pred, average="macro")
        f1_score_Acc += metrics.f1_score(y_test, y_pred, average="macro")
#        logLossAcc += metrics.log_loss(y_test, y_pred, eps=1e-15, normalize=False)
        
    print("Accuracy:",accAcum/epochs)
    print("precision:",precision_score_Acc/epochs)
    print("recall:",recall_score_Acc/epochs)
    print("f1:",f1_score_Acc/epochs)
#    print("logLoss:",logLossAcc/epochs)
    return y_test,y_pred,accAcum/epochs
#targetFeature = 'TypeOf'
epochs = 100
#y_test,y_pred,accuracy = RF(X,targetFeature,ratio,epochs)

Overall, very good scores using ``RandomForestClassifier``.  The following is ``confusion_matrix``of ``epochs = 100`` separate runs that averaged the results of ``RandomForestClassifier``.

In [ ]:
from sklearn.metrics import confusion_matrix
print(len(y_test))
confusion_matrix(y_test, y_pred)

## Augment the ``iris``dataset.

Now we can augment (increase by 10\% the amount data) with  synthetic data generated by a claas imbalence scheme,  ``SMOOT``. Just ``TypeOf=0`` needs 15(10%) rows (3 classes of 5 rows) of synthetic data to balance the classes. The result is similar to image augmention in that we accomplished structured data augmentation by increasing the row count ``Isis``dataset with synthetic data.

In [ ]:
from paso.pre.cleaners import Augment_by_Class
ratio = 0.1
targetFeature = 'TypeOf'
Class_BalancerKey = 'SMOTE'
X = Augment_by_Class(Class_BalancerKey).transform(Flower,targetFeature,0.1)
DataFrameSummary(X).summary()

Notice the ``mean``, ``std`` and other statistics have slightly changed. Also, the count has inreased by 10% or augmented a structured dataset by 10%. Why do expect slight and not large change in the dataset statistics?

In [ ]:
targetFeature = 'TypeOf'
epochs = 100
y_test,y_pred,accuracy = RF(X,targetFeature,ratio,epochs)

In [ ]:
print(len(y_test))
confusion_matrix(y_test, y_pred)

Hmmm, did not seem to help much. Let us try 100% augmentation or double the number of rows the  ``Flower`` dataframe.

In [ ]:
ratio = 1.00
targetFeature = 'TypeOf'
Class_BalancerKey = 'SMOTE'
X = Augment_by_Class(Class_BalancerKey).transform(Flower,targetFeature,ratio)
DataFrameSummary(X).summary()

In [ ]:
targetFeature = 'TypeOf'
epochs = 100
y_test,y_pred,accuracy = RF(X,targetFeature,ratio,epochs)

It seems to be more accurate in predicting the holdout (test) dataset.

In [ ]:
print(len(y_test))
confusion_matrix(y_test, y_pred)

Seemed to raise the accuracy by 0.024. Not bad, in both the real-world and the **Kaggle** world this is huge!

In [ ]:
ratio = 1.00
targetFeature = 'TypeOf'
Class_BalancerKey = 'SMOTE'
X = Augment_by_Class(Class_BalancerKey).transform(X,targetFeature,ratio)
DataFrameSummary(X).summary()

In [ ]:
targetFeature = 'TypeOf'
epochs = 100
y_test,y_pred,accuracy = RF(X,targetFeature,ratio,epochs)

In [ ]:
print(len(y_test))
confusion_matrix(y_test, y_pred)

Seemed to raise the accuracy by 0.014 or .039 by quadrupling.

In [ ]:
ratio = 1.00
targetFeature = 'TypeOf'
Class_BalancerKey = 'SMOTE'
X = Augment_by_Class(Class_BalancerKey).transform(X,targetFeature,ratio)
DataFrameSummary(X).summary()

In [ ]:
targetFeature = 'TypeOf'
epochs = 100
y_test,y_pred,accuracy = RF(X,targetFeature,ratio,epochs)

In [ ]:
print(len(y_test))
confusion_matrix(y_test, y_pred)

## Non-trivial class balancing

Let us rebalance and augment a larger dataset. [You can get ``otto_group``here.](https://www.kaggle.com/c/otto-group-product-classification-challenge/data). In this dataset we have about 62,000 rows and 8 different classes.

In [ ]:
otto_group_train = pd.read_csv("otto-group/train.csv")
DataFrameSummary(otto_group_train).summary()

The ``otto_group_train`` has a few majority(1,5) and minority classes(0,3,4.6).It is an unbalanced dataset.

In [ ]:
otto_group_train['target'] = otto_group_train['target'].astype('category').cat.codes
sns.set_color_codes()
sns.distplot(otto_group_train['target'],kde=False)

In [ ]:
otto_group_train[['id','target']].groupby(['target']).count()

In [ ]:
epochs = 10
targetFeature = 'target'
y_test,y_pred,accuracy = RF(otto_group_train,targetFeature,ratio,epochs)

In [ ]:
print(len(y_test))
confusion_matrix(y_test, y_pred)

 What happens when we class balance ``otto_group_train`` dataset ?

In [ ]:
ratio = 0.0
targetFeature = 'target'
Class_BalancerKey = 'SMOTE'
X = Augment_by_Class(Class_BalancerKey).transform(otto_group_train,targetFeature,ratio)
DataFrameSummary(X).summary()

We can use ``Augment_by_Class`` to balance ``otto_group_train``by augmenting by 0%.

In [ ]:
epochs = 10
y_test,y_pred,accuracy = RF(X,targetFeature,ratio,epochs)

In [ ]:
print(len(y_test))
confusion_matrix(y_test, y_pred)

In [ ]:
ratio = 1.00
targetFeature = 'target'
Class_BalancerKey = 'SMOTE'
X = Augment_by_Class(Class_BalancerKey).transform(X,targetFeature,ratio)
DataFrameSummary(X).summary()

In [ ]:
epochs = 10
y_test,y_pred,accuracy = RF(X,targetFeature,ratio,epochs)

In [ ]:
print(len(y_test))
confusion_matrix(y_test, y_pred)

Wait, maybe the accuracy is even better. We tripled the size of the dataset while also increasing the accuracy. What happens to a balanced ``otto_group_train`` dataset that is the same size as the original ``otto_group_train`` dataset?

## Summary

In summary paso data cleaning transformation classes are:
- ``Impute_Features_by_Values``

You have seen **paso** offers data cleaning classes for both production data engineers and research data scientists. **paso** support streaming data as well as bulk extraction data cleaning. You can expect **paso** to continue to offer state-of-the-art tools for data cleaning.

Other lessons on **paso** are:
1. [**paso**'s Offering of Logging and Parameter Services for your Python Project](https://github.com/bcottman/paso/blob/master/lessons/lesson_1.ipynb)
1. [**paso**'s Offering of Data Cleaners for your Machine or Deep Learning Projects](https://github.com/bcottman/paso/blob/master/lessons/lesson_2.ipynb)

In the future, we will cover **paso** in more depth with the following lesons:
- Overview of **paso** scalers and handling data outliers.
- Overview of **paso** encoders.
- Overview of **paso** machine learning and deep learning models.
- Using  **paso** on GPUs.
- and yet more…

If you have a service or feature or see a bug, then leave the **paso** project a [note](https://github.com/bcottman/paso/issues).